In [7]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

In [ ]:
import gdown
import zipfile
import os

# === 1. LINK DO PLIKU Z GOOGLE DRIVE ===
# Udostępnij plik publicznie i użyj ID z linku: https://drive.google.com/file/d/FILE_ID/view
file_id = '1n9aHRA-k3ElGwbmysJMjo8DoaMFNvxMg'  # <-- zamień na swoje ID
output_zip = 'dataset.zip'

# === 2. Pobierz plik ZIP ===
gdown.download(f'https://drive.google.com/uc?id={file_id}', output_zip, quiet=False)

# === 3. Rozpakuj (jeśli ZIP) ===
with zipfile.ZipFile(output_zip, 'r') as zip_ref:
    zip_ref.extractall('dataset')

print("✅ Dane pobrane i rozpakowane do folderu 'dataset/'")

In [2]:
class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target

In [3]:
def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )

train_dataset = get_coco_dataset(
    img_dir="dataset/train",
    ann_file="dataset/train/_annotations.coco.json"
)

val_dataset = get_coco_dataset(
    img_dir="dataset/valid",
    ann_file="dataset/valid/_annotations.coco.json"
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [5]:
num_classes = 6
model = get_model(num_classes)

C:\Users\Sebastian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Sebastian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\Sebastian/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:04<00:00, 37.0MB/s] 


In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [11]:
def train_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    for images, targets in tqdm(data_loader, desc=f"Trenowanie epoki: {epoch}"):
        images = [img.to(device) for img in images]

        processed_targets = []
        valid_images = []
        for i, target in enumerate(targets):
            boxes = []
            labels= []
            for obj in target:
                bbox = obj["bbox"]
                x, y, w, h = bbox

                if w>0 and h>0:
                    boxes.append([x, y, x+w, y+h])
                    labels.append(obj["category_id"])
                
            if boxes:
                processed_target = {
                    "boxes": torch.tensor(boxes, dtype=torch.float32).to(device),
                    "labels": torch.tensor(labels, dtype=torch.int64).to(device),
                }
                processed_targets.append(processed_target)
                valid_images.append(images[i])

        if not processed_targets:
            continue

        images = valid_images

        loss_dict = model(images, processed_targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch}] Loss: {losses.item():.4f}")

In [12]:
def training_loop(n_of_epochs):
    for epoch in range(n_of_epochs):
        train_epoch(model, optimizer, train_loader, device, epoch)
        lr_scheduler.step()
    
        # Save the model's state dictionary after every epoch
        model_path = f"fasterrcnn_resnet50_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Model saved: {model_path}")
